In [1]:
import polars as pl
import numpy as np
import pandas as pd
import os




In [2]:
import numpy as np
import pandas as pd
import os

from numba import njit
@njit()
def hmean(arr):
    return len(arr)/np.sum(1/arr)

@njit()
def gmean(arr):
    return np.exp(np.mean(np.log(arr)))

@njit()
def check_limit_val(profit, valuation, new_profit, limit_val_new):
    v_profit = new_profit.copy()
    for i in range(len(new_profit)-1, -1, -1):
        if valuation[i] > limit_val_new:
            v_profit[i] = profit[i]
    if gmean(v_profit) >= gmean(new_profit):
        return True
    return False
    
@njit()
def value_limit_drop(congthuc, profit, valuation):
    new_profit = np.array([profit[0]])
    limit_val = np.zeros(5)
    if profit[0] > 1.03:
        limit_val[-1] = valuation[0]
    else:
        limit_val[-1] = valuation[0]/(profit[0]**2)
    for id in range(1, len(profit)):
        #xử lí lợi nhuân
        if valuation[id] < limit_val[-1]:
            new_profit = np.append(new_profit, 1.03)
        else:
            new_profit = np.append(new_profit, profit[id])
        #xử lí ngưỡng
        if valuation[id] == 0:
            valuation[id] = gmean(valuation[: id])
        if profit[id] > 1.03 and valuation[id] <= limit_val[-1]:
            check = check_limit_val(profit, valuation, new_profit, gmean(np.append(limit_val, valuation[id]))/gmean(profit[: id])/profit[id])
            if check == True:
                limit_val = np.append(limit_val, gmean(np.append(valuation[: id+1], limit_val[0]))/gmean(profit[:id+1])/profit[id])
            else:
                limit_val = np.append(limit_val, gmean(np.append(valuation[: id+1], limit_val[0]))/gmean(profit[id:]))
        else:
            limit_val = np.append(limit_val, gmean(valuation[: id+1])/gmean(profit[: id+1]))
    # return [limit_val,gmean(new_profit[1:]), gmean(new_profit)]
    return limit_val

@njit()
def value_limit(congthuc, profit, valuation):
    profit_bank = 1.06
    new_profit = np.array([profit[0]])
    limit_val = np.full(5, np.inf)
    if profit[0] > profit_bank:
        limit_val[-1] = valuation[0]
    else:
        limit_val[-1] = valuation[0]/(profit[0]**2)
    for id in range(4, len(profit)-1):
        #xử lí lợi nhuân
        if valuation[id] < limit_val[-1]:
            new_profit = np.append(new_profit, profit_bank)
        else:
            new_profit = np.append(new_profit, profit[id])
        #xử lí ngưỡng
        if valuation[id] == 0:
            valuation[id] = gmean(valuation[: id])
        if profit[id] > profit_bank and valuation[id] <= limit_val[-1]:
            check = check_limit_val(profit, valuation, new_profit, gmean(np.append(limit_val, valuation[id]))/gmean(profit[: id])/profit[id])
            if check == True:
                limit_val = np.append(limit_val, gmean(np.append(valuation[: id+1], limit_val[0]))/gmean(profit[:id+1])/profit[id])
            else:
                limit_val = np.append(limit_val, gmean(np.append(valuation[: id+1], limit_val[0]))/gmean(profit[id:]))
        else:
            limit_val = np.append(limit_val, gmean(valuation[: id+1])/gmean(profit[: id+1]))
        # print(valuation[id], limit_val[-2:], profit[id])
    # # return [limit_val,gmean(new_profit[1:]), gmean(new_profit)]
    # print(profit)
    # print(limit_val)
    # print(valuation)
    return limit_val



from complete_method import CompleteMethod
file_path = 'Data_YearFromQuarter.csv'
data = pd.read_csv(file_path)
# data = pd.read_csv('/content/drive/MyDrive/November/POWER_METHOD/QUARTER/data_test_not_invest.csv')
pathSaveFormula = "./test"
if not os.path.exists(pathSaveFormula):
    os.mkdir(pathSaveFormula)

NUMBER_QUARTER = 62

vis = CompleteMethod(data, pathSaveFormula, NUMBER_QUARTER, "geomean")

vis.generate_formula(1, 10000, 10000)



print('DONE')













Các cột không được coi là biến: ['TIME', 'PROFIT', 'SYMBOL', 'EXCHANGE']. 
Khi ngắt bằng tay thì cần tự chạy phương thức <CompleteMethod_object>.save_history() để lưu lịch sử. 
Đang chạy sinh công thức có số toán hạng là  1 . . .
Đã lưu lịch sử. 
Đã lưu công thức 
Đang chạy sinh công thức có số toán hạng là  2 . . .
Đã lưu lịch sử. 
Đã lưu công thức 
Đang chạy sinh công thức có số toán hạng là  3 . . .
Đã lưu lịch sử. 
Đã lưu công thức 


Exception: Đã sinh đủ công thức theo yêu cầu.

In [3]:

df_fomula = vis.convert_npy_file_to_DataFrame('./test/high_profit_2_28_02_2023_12_06_45.npy')

df = df_fomula[df_fomula.invest != 'NI'].reset_index(drop= True)
df

,formula,geomean_profit,invest,profit
0,+0*0,0.0,VIC,0.0
1,+0*1,0.0,VIC,0.0
2,+0*2,0.0,VIC,0.0
3,+0*3,0.0,VIC,0.0
4,+0*4,0.0,VIC,0.0
...,...,...,...,...
9546,+22+70,0.0,VHM,0.0
9547,+22+71,0.0,VHM,0.0
9548,+22+72,0.0,NVL,0.0
9549,+22+73,0.0,VHM,0.0


In [3]:

df_fomula = vis.convert_npy_file_to_DataFrame('./test/high_profit_3_28_02_2023_12_22_16.npy')

df = df_fomula[df_fomula.invest != 'NI'].reset_index(drop= True)


df_fomula = df_fomula[2000:3000].reset_index(drop= True)

arr_congthuc = df_fomula.formula
arr_profit = np.zeros((1000, NUMBER_QUARTER))
arr_value = np.zeros((1000, NUMBER_QUARTER))
arr_limit = np.zeros((1000, NUMBER_QUARTER))
arr_rank_not_invest = np.zeros((1000, NUMBER_QUARTER))


for i in range(len(df_fomula)):
    temp = vis.explain_formula(df.formula.iloc[i], not_invest_available= True)
    arr_profit[i] = np.array(temp.Out_Profit)
    arr_value[i] = np.array(temp.Out_Value)
    arr_rank_not_invest[i] = np.array(temp.Out_Rank_NI)
    arr_limit[i] = value_limit(df_fomula.formula.iloc[i], np.array(temp.Out_Profit), np.array(temp.Out_Value))

np.save('./congthuc/congthuc.npy', arr_congthuc, allow_pickle= True)
np.save('./congthuc/all_profit.npy', arr_profit, allow_pickle= True)
np.save('./congthuc/all_value.npy', arr_value, allow_pickle= True)
np.save('./congthuc/all_limit.npy', arr_limit, allow_pickle= True)
np.save('./congthuc/arr_rank_not_invest.npy', arr_rank_not_invest, allow_pickle= True)

In [4]:

for i in range(1):
    temp = vis.explain_formula(df.formula.iloc[i], not_invest_available= True)
    arr_profit[i] = np.array(temp.Out_Profit)
    arr_value[i] = np.array(temp.Out_Value)
    arr_rank_not_invest[i] = np.array(temp.Out_Rank_NI)
    print(np.array(temp.Out_Profit), np.array(temp.Out_Value))
    arr_limit[i] = value_limit(df_fomula.formula.iloc[i], np.array(temp.Out_Profit), np.array(temp.Out_Value))


[0.76510417 0.73882979 0.62941176 0.47159091 0.61357143 0.775
 0.87433628 0.92811839 1.04516129 0.83333333 1.30496454 1.07738095
 1.14705882 1.80280672 1.27868852 1.29173333 1.22556701 0.81727273
 0.93786058 0.73046875 0.95922119 1.03149327 1.07181529 1.13814103
 1.72786177 1.84684685 1.75877863 1.68527132 1.06493506 0.69387755
 0.53448276 0.48269231 1.2606823  1.33384615 1.3294186  1.28889823
 1.27009604 1.0489499  1.21677852 1.37076023 2.04245283 3.09405941
 2.8958427  1.91       1.47408138 1.10317817 1.11569507 1.27870564
 1.10384615 0.81818182 0.7312187  0.76358149 0.86538462 1.40631579
 1.3580402  1.4675033  1.14424498 0.86474164 0.72080773 0.57038835
 0.608273   0.02911208] [1.20627715e+38 1.26636260e+38 1.03847738e+38 1.11919425e+38
 1.22884055e+38 1.83609172e+38 1.91732255e+38 1.26691016e+38
 5.36071639e+37 8.58034574e+37 1.62868774e+38 2.29153506e+38
 2.22087400e+38 1.86673191e+38 1.83139412e+38 2.37439136e+38
 4.70301345e+38 1.04251609e+39 8.95169792e+38 7.90576687e+38
 2.742

In [5]:
data

,SYMBOL,EXCHANGE,TIME,PROFIT,MARKET_CAP,CURRENT ASSETS,Cash and cash equivalents,Cash,Cash equivalents,Short Term Financial Investments,...,General and administration expenses,Net operating profit,Other income,Other expenses,Other profit,Total accounting profit before tax,Current income tax,Deferred income tax,Profit after tax,Net profit after tax of the parent
0,NOT_INVEST,HOSE,92,1.000000,9.468467e+12,4.986414e+12,7.157485e+11,2.840622e+11,4.313711e+11,5.841027e+11,...,2.358154e+11,7.351276e+11,5.406774e+10,3.424368e+10,1.980027e+10,7.571205e+11,1.582448e+11,4.895160e+08,5.988736e+11,5.685004e+11
1,AAA,HOSE,92,0.000000,3.100000e+12,5.623485e+12,1.655457e+12,1.228607e+12,4.268494e+11,4.361560e+11,...,2.473068e+11,1.839187e+11,7.323958e+09,9.680181e+09,-2.356222e+09,1.815617e+11,7.265925e+10,-3.853180e+09,1.127560e+11,1.975472e+11
2,AAT,HOSE,92,0.000000,3.480000e+11,4.472210e+11,1.581258e+10,1.581258e+10,0.000000e+00,1.010230e+09,...,1.736932e+10,1.135980e+11,6.584209e+08,1.079498e+09,-4.210768e+08,1.131770e+11,2.285129e+10,0.000000e+00,9.032568e+10,9.032568e+10
3,ABS,HOSE,92,0.000000,4.840000e+11,1.020679e+12,3.583126e+09,3.583126e+09,0.000000e+00,0.000000e+00,...,1.018778e+10,3.955079e+10,3.509512e+08,1.013317e+09,-6.623654e+08,3.888842e+10,1.184588e+10,0.000000e+00,2.704254e+10,2.704254e+10
4,ACL,HOSE,92,0.000000,5.870000e+11,1.206780e+12,3.806691e+10,3.806691e+10,0.000000e+00,0.000000e+00,...,4.368854e+10,1.372759e+11,9.622188e+07,2.055397e+09,-1.959175e+09,1.353167e+11,1.618693e+10,4.108935e+08,1.187189e+11,1.187189e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7390,AGF,HOSE,28,0.597410,1.096373e+12,2.746470e+11,1.295624e+10,1.295624e+10,0.000000e+00,2.452173e+10,...,1.577683e+10,5.068815e+10,1.982435e+09,2.179126e+09,-1.966909e+08,5.049146e+10,5.049146e+09,0.000000e+00,4.544232e+10,4.544232e+10
7391,BMP,HOSE,28,0.564000,3.483350e+12,3.461020e+11,1.022540e+11,1.022540e+11,0.000000e+00,0.000000e+00,...,1.528187e+10,1.084229e+11,5.770233e+08,5.629300e+05,5.764604e+08,1.089993e+11,5.513941e+09,0.000000e+00,1.034854e+11,1.034854e+11
7392,VSH,HOSE,28,0.524706,1.062500e+13,1.583840e+11,7.275869e+10,7.275869e+10,0.000000e+00,0.000000e+00,...,5.369494e+09,2.085103e+11,0.000000e+00,0.000000e+00,0.000000e+00,2.085103e+11,0.000000e+00,0.000000e+00,2.085103e+11,2.085103e+11
7393,TYA,HOSE,28,0.518450,1.713390e+12,8.734550e+11,3.927625e+10,3.927625e+10,0.000000e+00,0.000000e+00,...,1.996985e+10,4.659002e+10,9.592896e+09,1.734758e+08,9.419420e+09,5.600944e+10,3.535409e+09,0.000000e+00,5.247403e+10,5.247403e+10


In [7]:
data[(data.PROFIT < 0.03 ) & (data.PROFIT > 0 )]

,SYMBOL,EXCHANGE,TIME,PROFIT,MARKET_CAP,CURRENT ASSETS,Cash and cash equivalents,Cash,Cash equivalents,Short Term Financial Investments,...,General and administration expenses,Net operating profit,Other income,Other expenses,Other profit,Total accounting profit before tax,Current income tax,Deferred income tax,Profit after tax,Net profit after tax of the parent
346,CNG,HOSE,90,0.027397,9.855000e+11,1.113749e+12,3.883570e+11,3.635699e+10,3.520000e+11,3.428800e+10,...,8.643880e+10,1.487905e+11,1.496703e+09,1.136444e+08,1.383058e+09,1.501735e+11,3.529373e+10,-3.895486e+09,1.187753e+11,1.187753e+11
347,VSC,HOSE,90,0.026008,4.662808e+12,1.228246e+12,5.826776e+11,1.168531e+11,4.658245e+11,2.550189e+11,...,8.924142e+10,5.475680e+11,1.318370e+10,1.768452e+10,-4.500819e+09,5.430669e+11,8.698217e+10,1.128991e+08,4.559716e+11,3.842795e+11
348,RAL,HOSE,90,0.025773,2.225900e+12,5.891501e+12,9.453468e+11,9.453468e+11,0.000000e+00,3.000000e+11,...,1.835910e+11,5.367434e+11,1.112891e+09,1.779033e+09,-6.661419e+08,5.360768e+11,1.101385e+11,0.000000e+00,4.259380e+11,4.259380e+11
349,NT2,HOSE,90,0.024119,7.758259e+12,4.475436e+12,2.049195e+11,2.044287e+11,4.907904e+08,0.000000e+00,...,7.773720e+10,9.679858e+11,3.046427e+09,1.117812e+09,1.928616e+09,9.699143e+11,6.167293e+10,-1.069771e+10,9.189395e+11,9.189395e+11
350,VHC,HOSE,90,0.023256,1.577042e+13,8.018053e+12,5.299736e+10,4.499736e+10,8.000000e+09,1.659335e+12,...,2.402753e+11,2.384582e+12,2.840365e+10,3.573354e+10,-7.329884e+09,2.377253e+12,3.226888e+11,-4.236382e+09,2.058800e+12,2.041941e+12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
623,PTB,HOSE,89,0.004717,5.151541e+12,3.204400e+12,2.043650e+11,2.043650e+11,0.000000e+00,3.904716e+10,...,2.021049e+11,6.640060e+11,2.768776e+10,1.968996e+10,7.997803e+09,6.720040e+11,1.318498e+11,5.334848e+08,5.443040e+11,5.300578e+11
624,FRT,HOSE,89,0.003524,1.120752e+13,1.015070e+13,9.191220e+11,6.779950e+11,2.411270e+11,1.611500e+12,...,7.512620e+11,7.110068e+11,1.045248e+10,2.038672e+09,8.413805e+09,7.194209e+11,1.349792e+11,2.116454e+09,5.823253e+11,5.773879e+11
6472,NKD,HOSE,42,0.028916,6.275895e+11,2.232710e+11,3.980114e+10,3.980114e+10,0.000000e+00,4.878477e+10,...,3.623427e+10,8.956662e+10,4.152575e+09,3.110773e+09,1.041802e+09,9.701590e+10,1.766340e+10,0.000000e+00,7.294502e+10,7.897435e+10
6613,NKD,HOSE,41,0.025316,7.168130e+11,2.120790e+11,3.194783e+10,3.194783e+10,0.000000e+00,5.686161e+10,...,2.856603e+10,1.110999e+11,3.545160e+09,3.502400e+09,4.275990e+07,1.111426e+11,2.151051e+10,0.000000e+00,8.963214e+10,8.958065e+10
